# Eager Mode Mnist

In [1]:
from __future__ import absolute_import, division, print_function
import tensorflow as tf
tf.enable_eager_execution()

In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="1"
print(tf.__version__)

2.0.0-alpha0


mnist image : 28,28 

In [3]:
mnist = tf.keras.datasets.mnist

# Create Model

In [4]:

class MnistBasic(tf.keras.Model):
    def __init__(self):
        super(MnistBasic, self).__init__()
        self.conv1 = tf.keras.layers.Conv2D(16, kernel_size=3, activation='relu')
        self.conv2 = tf.keras.layers.Conv2D(16, kernel_size=3, activation='relu')
        self.pool  = tf.keras.layers.GlobalAveragePooling2D()
        self.fc    = tf.keras.layers.Dense(10)
        
    def call(self, input):
        output = self.conv1(input)
        output = self.conv2(output)
        output = self.pool(output)
        output = self.fc(output)
        return output 
    
# def create_model():
#     model = tf.keras.Sequential()
#     # Build the model
#     model.add(tf.keras.layers.Conv2D(16,[3,3], activation='relu'))
#     model.add(tf.keras.layers.Conv2D(16,[3,3], activation='relu'))
#     model.add(tf.keras.layers.GlobalAveragePooling2D())
#     model.add(tf.keras.layers.Dense(10))
#     return model 

In [5]:
# def create_model2():
#     model = tf.keras.Sequential()
#     # Build the model
#     model.add(tf.keras.layers.Conv2D(16,[3,3], activation='relu', use_bias=False))
#     model.add(tf.keras.layers.BatchNormalization())
#     model.add(tf.keras.layers.Conv2D(16,[3,3], activation='relu', use_bias=False))
#     model.add(tf.keras.layers.BatchNormalization())
#     model.add(tf.keras.layers.GlobalAveragePooling2D())
#     model.add(tf.keras.layers.Dense(10))
#     return model 

In [6]:
# model = create_model2()
model = MnistBasic()
print(model)

# Load Model 

In [9]:
# Fetch and format the mnist data
(mnist_images, mnist_labels), _ = tf.keras.datasets.mnist.load_data()

dataset = tf.data.Dataset.from_tensor_slices(
  (tf.cast(mnist_images[...,tf.newaxis]/255, tf.float32),
   tf.cast(mnist_labels,tf.int64)))
dataset = dataset.shuffle(1000).batch(32)

In [10]:
for images, labels in dataset.take(1):
    print(images.shape)
    output = model(images)
    print("Logis", output.shape)
    

(32, 28, 28, 1)
Logis (32, 10)


# Define Optimizer

In [11]:
print(tf.__version__)

2.0.0-alpha0


In [15]:
loss_fn = tf.losses.sparse_softmax_cross_entropy
optimizer = tf.keras.optimizers.Adam(1e-4)

# Train

In [16]:
def loss(model, inputs, targets):
    error = tf.sparse.softmax(model(inputs)) - targets
    return tf.reduce_mean(tf.square(error))

In [17]:
epoch=1
loss_history = []
acc_history = []

@tf.function
def train_step(inputs, labels):
#     accuracy = tf.metrics.Accuracy('accuracy', dtype=tf.float16)
    
    with tf.GradientTape() as tape :
        predictons = model(inputs)
#         accuracy(tf.argmax(predictons, axis=1, output_type=tf.int8), tf.cast(labels, tf.int8))
        regular_loss = loss(model, inputs, labels)
        pred_loss = loss_fn(labels, predictons)
        total_loss = pred_loss + regular_loss
    
    gradients = tape.gradient(total_loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

for _ in range(epoch):
    for ( batch, (images, labels)) in enumerate(dataset.take(2000)):
        train_step(images, labels)
        # accuracy = tf.metrics.Accuracy('accuracy', dtype=tf.float32)
        # with tf.GradientTape() as tape:
        #     logits = model(images, training=True)
        #     loss_value = loss_fn(labels, logits)
        #     accuracy(tf.argmax(logits, axis=1, output_type=tf.int64),
        #             tf.cast(labels, tf.int64))
        # 
        # loss_history.append(loss_value.numpy())
        # acc_history.append(accuracy.result())
        # grads = tape.gradient(loss_value, model.variables)
        # optimizer.apply_gradients(zip(grads, model.variables))
        # print(images.shape)
        # print(labels.shape)
        
    
        # if batch % 20== 0 :
#             print("Loss at step {:03d}: {:.3f}, Accu : {:.3f}".format(batch, loss_value, accuracy.result()))
#             print("Loss at step {:03d}: ".format(batch))
#         if batch > 1860 :
#             print(batch)
        

AttributeError: 'Tensor' object has no attribute 'indices'

In [ ]:
import matplotlib.pyplot as plt
plt.plot(loss_history)
plt.plot(acc_history)
plt.xlabel('Batch #')
plt.ylabel('Loss [entropy]')
plt.show()